In [15]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import pytz
import yfinance as yf

import warnings
warnings.filterwarnings("ignore")

In [16]:
tolerance = 0.
k = 12
h = 2

# rule = 'BME'

time_range = '5y'

start_time = '2019-12-31'
end_time = '2024-01-01'

# Vol scaling
vol_flag = 1               # Set flag to 1 for vol targeting
if vol_flag == 1:
    target_vol = 0.4 # Can be changed
else:
    target_vol = 'no target vol'


In [17]:
# For S$P 500 stock

# tickers = pd.read_html( 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

# # stock_list = tickers['Symbol'].to_list()
# print(tickers)
# stock_list = tickers['Symbol'][:50]

In [18]:
def US_Stock_data():

    # Euro Stock
    stock_list = pd.read_html( 'https://en.wikipedia.org/wiki/EURO_STOXX_50')[4]['Ticker'][1:].to_list()
    futures = pd.DataFrame(columns= stock_list)

    time_index = list(yf.Ticker(stock_list[0]).history(period = time_range,start = start_time, end = end_time).index)
    flag = 0

    for symbol in stock_list:
        df = yf.Ticker(symbol).history(period = time_range, start = start_time, end = end_time)
        df = pd.DataFrame(df['Close'])
        i = 0
        daily_return = []
        for k in df['Close']:
            if i != 0:
                daily_return.append(float((k-i)/i))
            else:
                daily_return.append(float(0))
            i = k
        if time_index != list(df.index): flag +=1
        try:
            futures[symbol] = daily_return
        except:
            # print(symbol) #Bugfix purpose
            # print(len(daily_return)) #Bugfix purpose
            while len(daily_return) < len(futures):
                daily_return.insert(0,float(0))
            futures[symbol] = daily_return

    futures.index = time_index
    futures = futures.iloc[1:, :]


    # print(flag) #Bugfix

    futures['Date'] = pd.to_datetime(futures.index, format='%Y-%m-%d')
    futures.set_index('Date', inplace=True)

    return futures

In [19]:
def Volatility_scale(data):

    std_index = data.index

    daily_index = pd.DataFrame(index=std_index)

    for oo in data.columns:
        returns = data[oo]
        returns.dropna(inplace=True)
    
        ret_index = (1 + returns).cumprod()
        ret_index[0] = 1
    
        # ex ante vol estimate
        day_vol = returns.ewm(ignore_na=False,
                          adjust=True,
                          com=60,   
                          min_periods=0).std(bias=False)
        vol = day_vol * np.sqrt(261) # annualise
    
        ret_index = pd.concat([ret_index, vol], axis=1)
        ret_index.columns = [oo, oo + '_Vol']

        daily_index = pd.concat([daily_index, ret_index], axis=1)

    return daily_index

In [20]:
def backtest_strategy(data,k = k, h = h):

    pnl = pd.DataFrame(index=data.index)


    daily_index = Volatility_scale(data)

    company = data.columns


    for oo in company:
        flag = signal = 0
        df = pd.concat([daily_index[oo], daily_index[oo+"_Vol"]], axis=1)
        df['returns'] = df[oo].pct_change(k)
        df['pnl'] = 0.
        try:
            for x, v in enumerate(df['returns']):
                # adjust return with the holding time
                if flag != 0:
                    flag = flag - 1
                    if signal == 1:
                        if vol_flag == 1:
                            if df[oo+"_Vol"].iloc[x - ((h-1) - flag)] == 0:
                                df['pnl'].iloc[x] = 0
                            else:
                                df['pnl'].iloc[x] = (df[oo].iloc[x - 1] / df[oo].iloc[x] - 1) * (target_vol / df[oo+"_Vol"].iloc[x - ((h-1) - flag)])
                        else: df['pnl'].iloc[x] = (df[oo].iloc[x - 1] / df[oo].iloc[x] - 1)
                    elif signal == -1:
                        if vol_flag == 1:
                            if df[oo+"_Vol"].iloc[x - ((h-1) - flag)] == 0:
                                df['pnl'].iloc[x] = 0
                            else:
                                df['pnl'].iloc[x] = (df[oo].iloc[x] / df[oo].iloc[x - 1] - 1) * (target_vol / df[oo+"_Vol"].iloc[x - ((h-1) - flag)])
                        else: df['pnl'].iloc[x] = (df[oo].iloc[x] / df[oo].iloc[x - 1] - 1)
                    continue
                if x <= k:
                # skip the first k day(s) observations
                    continue
                if df['returns'].iloc[x-1] < tolerance:
                # negative returns, sell and hold for h day, then close position
                    if vol_flag == 1:
                        df['pnl'].iloc[x + h -1] = (df[oo].iloc[x - 1] / df[oo].iloc[x+h-1] - 1) * \
                                        target_vol / df[oo+"_Vol"].iloc[x -1]
                    else:
                        df['pnl'].iloc[x + h -1] = (df[oo].iloc[x - 1] / df[oo].iloc[x+h -1] - 1)
                    signal = -1
                elif df['returns'].iloc[x-1] > tolerance:
                # positive returns, buy and hold for h day, then close position
                    if vol_flag == 1:
                        df['pnl'].iloc[x + h -1] = (df[oo].iloc[x +h -1] / df[oo].iloc[x -1] - 1) * \
                                        target_vol / df[oo+"_Vol"].iloc[x -1]
                    else:
                        df['pnl'].iloc[x + h - 1] = (df[oo].iloc[x + h -1] / df[oo].iloc[x -1] - 1)
                    
                    signal = 1
                flag = h - 1
        except: pass
        # convert to cummulative index
        pnl = pd.concat([pnl, df['pnl']], axis=1)

    pnl.columns = data.columns
    df = pnl
    df['port_avg'] = df.mean(skipna = False, axis=1)

    return df['port_avg']

In [21]:
def strategy_stat(data):
    return [data.mean(), data.std(), data.mean()/data.std(), data.min()]

In [ ]:
data = US_Stock_data()

for m in [1,3,5,7,15,20,30,60]:
    for n in  [1,3,5,7,15,20,30,60]:
        print([m,n])
        temp = backtest_strategy(data,m,n)
        try:
          temp2 = temp.to_list()
          temp2.insert(0,n)
          temp2.insert(0,m)
          stats.loc[len(stats.index)] = temp2
        except:
          index = temp.index.to_list()
          index.insert(0,'h')
          index.insert(0,'k')
          stats = pd.DataFrame(columns = index)
          temp2 = temp.to_list()
          temp2.insert(0,n)
          temp2.insert(0,m)
          stats.loc[len(stats.index)] = temp2

stats.to_csv("k_h.csv")
del stats